In [ ]:
import os
import time
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

from joblib import load

import raster_to_features as rf

import geopandas as gpd
from shapely.geometry import mapping

In [10]:
# year for which to predict iceplant locations
year = 2012

# whether to save rasters
save_rasters = True

In [ ]:
# open shapefile of SB coastal buffer and process it to use it for clipping
fp = os.path.join(os.getcwd(), 'separating_naip_flights', 'SB_coastal_buffer', 'SB_coastal_buffer.shp')
coast = gpd.read_file(fp)
coast_geo = coast.geometry.apply(mapping)

In [ ]:
# open pre-trained random forest classifier
rfc = load('spectral_rfc.joblib') 
print('loaded model')

In [ ]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(30)

client = cluster.get_client()
client

In [ ]:
# select the scene ids from given year that intersect the coastal buffer
# the itemids of all scenes that intersect the coast were previously stored in a csv
scene_ids = pd.read_csv(os.path.join(os.getcwd(),
                                     'separating_naip_flights',
                                     'coastal_scenes_ids.csv'))
scene_ids = scene_ids.loc[scene_ids['year'] == year]
scene_ids = scene_ids.reset_index().itemid

In [ ]:
# ---------------------------------------
# collect processing information for each scene
times = []
processed = []
reason = []

# counter for scenes queued for processing
N = len(scene_ids)

# ---------------------------------------
# prepare folder to save rasters
if save_rasters:
    fp = os.path.join(os.getcwd(),'temp')
    if os.path.exists(fp) == False:
        os.mkdir(fp)
    fp = os.path.join(fp, 'S_clip_preds_' + str(year))
    if os.path.exists(fp) == False:
        os.mkdir(fp)

        
# ---------------------------------------
# ---------------------------------------
for itemid in scene_ids:
    t_alpha = time.time()

    # ---------------------------------------
    # open NAIP scene and clip to coast
    t0 = time.time()
    raster = rf.rioxr_from_itemid(itemid).rio.clip(coast_geo,coast.crs)
    print('clipped raster', time.time() - t0,' s')

    # ---------------------------------------
    # select pixels with data (blacked out portions have 0 on all bands)
    df = rf.raster_as_df(raster.to_numpy(), ['r','g','b','nir'])
    df = df.loc[ (df['nir'] != 0) | (df['r'] != 0) | (df['g'] != 0) | (df['b'] != 0)]
    print('selected non-zero pixels')

    # ---------------------------------------
    # stop if there's no data at intersection
    if df.shape[0] == 0:
        processed.append('N')
        reason.append('no data in intersection')
        times.append(0)
        print('no data at intersection of scene with coastal buffer')
        print('FINISHED: ', itemid , '\n')

    else:
        # find vegetation pixels to go into model
        # keep indices of water and low-ndvi pixels
        # add ndvi and ndwi features for each pixel
        t0 = time.time()
        is_veg, water_index, not_veg_index = rf.add_spectral_features(df, 
                                                                      ndwi_thresh = 0.3, 
                                                                      ndvi_thresh = 0.05) 
        # ---------------------------------------
        # stop if there are no vegetation pixels at intersection
        if is_veg.shape[0] == 0:
            print('no vegetation pixels at intersection of scene data with coastal buffer')
            processed.append('N')
            reason.append('no vegeatation in intersection')
            times.append(0)
            print('FINISHED: ', itemid , '\n')
            
        else:
            processed.append('Y')
            reason.append('processed')            
            
            # ---------------------------------------
            # select features
            is_veg.drop('ndwi', axis=1, inplace=True)
            is_veg = rf.add_date_features(is_veg, rf.rioxr_from_itemid(itemid).datetime)
            print('assembled pixels dataframe with features\n   time taken to assemble: ', time.time() - t0,' s')

            # ---------------------------------------
            # convert into dask.array and predict using model
            da_pixels = da.from_array(np.array(is_veg), chunks=728802)

            scene_preds = rfc.predict(da_pixels)    

            t0 = time.time()
            preds = scene_preds.compute()
            print('finished predicting\n   time taken to predict: ', time.time() - t0,' s')

            # ---------------------------------------
            # recover pixel indices for iceplant classifications
            preds_df = pd.DataFrame(preds, 
                                 columns=['is_iceplant'], 
                                 index = is_veg.index)
            is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
            non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

            # ---------------------------------------
            # reconstruct indices into image
            indices = [non_iceplant_index,
                       is_iceplant_index, 
                       not_veg_index,
                       water_index]
            values = [0,    # values assigned to pixels from each index
                      1,
                      2,
                      3]
            t0 = time.time()
            reconstruct = rf.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)
            print('reconstructed image\n   time taken to reconstruct: ', time.time() - t0,' s')

            # ---------------------------------------
            # save raster 
            if save_rasters:                
                filename = 'S_clip_preds_' + itemid +'.tif'

                with rasterio.open(
                    os.path.join(fp, filename),  # file path
                    'w',           # w = write
                    driver = 'GTiff', # format
                    height = reconstruct.shape[0], 
                    width = reconstruct.shape[1],
                    count = 1,  # number of raster bands in the dataset
                    dtype = rasterio.uint8,
                    crs = raster.rio.crs,
                    transform = raster.rio.transform(),
                ) as dst:
                    dst.write(reconstruct.astype(rasterio.uint8), 1)
            # ---------------------------------------
            # print and append final stats
            total =  time.time() - t_alpha
            times.append(total)
            print('total time:', total)
            print('FINISHED: ', itemid)
            N = N-1
            print('REMAINING: ', N, 'scenes \n')

In [ ]:
#save times processed and itemids as dataframe
D = { 'itemid': scene_ids,
      'processed': processed,
    'reason':reason,
    'process_time': times}
processing_df = pd.DataFrame( D )

fp = os.path.join(os.getcwd(),'processing_results')
if os.path.exists(fp) == False:
    os.mkdir(fp)

processing_df.to_csv(os.path.join(fp, 'S_clip_processing_results_' + str(year) + '.csv'))